# Problem-Specific Coordinate Generation for HyperNEAT Substrates

# Extra Part: Additional Random Substrates

This part was added to validate the performance of random substrates, by generating more substrates with different seeds.

## Setup

### Imports

In [1]:
import pickle
from collections import defaultdict

from config import config

from tensorneat.algorithm.hyperneat import FullSubstrate
from substrate_generation.random_coor_generator import RandomCoordinateGenerator
from substrate_generation.hidden_layers import HiddenLayerGenerator

from evol_pipeline.brax_env import CustomBraxEnv

from utils.utils import setup_folders_substrate, save_coordinates_to_csv

The folder structure is checked to avoid problems when saving files.

In [2]:
OUTPUT_DIR = config["experiment"]["output_dir_substrates"]
setup_folders_substrate(OUTPUT_DIR)

The environment needs to be setup beforehand to get obs_size and act_size.

In [3]:
env_name = config["experiment"]["env_name"]
env_problem = CustomBraxEnv(
    env_name=env_name,
    backend=config["environment"]["backend"],
    brax_args=config["environment"]["brax_args"],
    max_step=config["environment"]["max_step"],
    repeat_times=config["environment"]["repeat_times"],
    obs_normalization=config["environment"]["obs_normalization"],
    sample_episodes=config["environment"]["sample_episodes"],
)
obs_size = env_problem.input_shape[0]
act_size = env_problem.output_shape[0]
feature_dims_repeats = config["data_analysis"]["feature_dims"]

/home/andi/anaconda3/envs/jax/lib/python3.10/site-packages/brax/io/mjcf.py:480: UserWarning: Brax System, piplines and environments are not actively being maintained. Please see MJX for a well maintained JAX-based physics engine: https://github.com/google-deepmind/mujoco/tree/main/mjx. For a host of environments that use MJX, see: https://github.com/google-deepmind/mujoco_playground.
  warnings.warn(
2025-11-07 07:28:43.788772: W external/xla/xla/service/gpu/autotuning/dot_search_space.cc:200] All configs were filtered out because none of them sufficiently match the hints. Maybe the hints set does not contain a good representative set of valid configs?Working around this by using the full hints set instead.
2025-11-07 07:28:54.454941: W external/xla/xla/service/gpu/autotuning/dot_search_space.cc:200] All configs were filtered out because none of them sufficiently match the hints. Maybe the hints set does not contain a good representative set of valid configs?Working around this by usin

In [4]:
feature_dims_repeats = config["data_analysis"]["feature_dims"]

def deep_defaultdict():
    return defaultdict(deep_defaultdict)

generated_io_coors = deep_defaultdict()
substrates = deep_defaultdict()

In [5]:
hidden_layer_gen = HiddenLayerGenerator(
    env_name=env_name,
    obs_size=obs_size,
    act_size=act_size,
    hidden_layer_type=config["substrate"]["hidden_layer_type"],
    hidden_depth=config["substrate"]["hidden_depth"],
    depth_factor=config["substrate"]["depth_factor"],
    width_factor=config["substrate"]["width_factor"],
)

### Random Substrate

We also want to test if our data-driven approaches are better than arbritarily chosen coordinates in an equally high-dimensional space. For this purpose the generator creates a substrate with the given number of dimensions in which all coordinates are random binary combinations, e.g. (0,1,0,1,1,0). If the target number of coordinates can't be created from unique binary combinations, the generator uses terniary combinations of 0, 0.5, 1. If that also fails to generate the required number of coordinates, if falls back to continous random numbers.

In [6]:
seeds = [231,10319]

for seed in seeds:
    for feature_dim in feature_dims_repeats:  
        random_generator = RandomCoordinateGenerator(
            obs_size=obs_size,
            act_size=act_size,
            feature_dims=feature_dim,
            depth_factor=config["substrate"]["depth_factor"],
            width_factor=config["substrate"]["width_factor"],
            seed=seed,
        )
        input_coors, output_coors = random_generator.generate_io_coordinates()
        hidden_coors = hidden_layer_gen.get_hidden_coors(input_coors=input_coors)
        substrates["no_data"]["random_coors"][feature_dim][f"{seed}"]["input_coors"] = input_coors
        substrates["no_data"]["random_coors"][feature_dim][f"{seed}"]["hidden_coors"] = hidden_coors
        substrates["no_data"]["random_coors"][feature_dim][f"{seed}"]["output_coors"] = output_coors

        substrates["no_data"]["random_coors"][feature_dim][f"{seed}"]["substrate"] = FullSubstrate(
            input_coors=input_coors,
            hidden_coors=hidden_coors,
            output_coors=output_coors
        )

        log_coors = [*input_coors, *hidden_coors, *output_coors]
        save_coordinates_to_csv(
            coordinates=log_coors,
            filepath=f"{OUTPUT_DIR}/coordinates/no_data_random_coors_{feature_dim}_seed{seed}_io.csv",
        )

Generating random coordinates with 1 feature dimensions...
Added layering dimension. Final coordinate size: 2
Successfully saved coordinates to: output_substrates/coordinates/no_data_random_coors_1_seed231_io.csv
Generating random coordinates with 2 feature dimensions...
Added layering dimension. Final coordinate size: 3
Successfully saved coordinates to: output_substrates/coordinates/no_data_random_coors_2_seed231_io.csv
Generating random coordinates with 5 feature dimensions...
Added layering dimension. Final coordinate size: 6
Successfully saved coordinates to: output_substrates/coordinates/no_data_random_coors_5_seed231_io.csv
Generating random coordinates with 9 feature dimensions...
Added layering dimension. Final coordinate size: 10
Successfully saved coordinates to: output_substrates/coordinates/no_data_random_coors_9_seed231_io.csv
Generating random coordinates with 14 feature dimensions...
Added layering dimension. Final coordinate size: 15
Successfully saved coordinates to: 

### Substrate from Data Analysis Results

Next, the input and output nodes created through data analysis are used to create substrates.

The substrates are saved to disk to use in other notebooks.

In [7]:
with open("random_substrates.pkl", "wb") as f:
    pickle.dump(substrates, f)

In [8]:
import os

import wandb

from config import config

import jax
from evol_pipeline.custom_pipeline import CustomPipeline
from evol_pipeline.evol_algorithm import create_evol_algorithm

from utils.visualization import visualize_cppn, visualize_nn
from utils.utils import setup_folders_evolution, append_summary_row

In [9]:
OUTPUT_DIR = config["experiment"]["output_dir_evolution"]
setup_folders_evolution(OUTPUT_DIR)

In [11]:
for feature_dims_label, feature_dims_dict in substrates["no_data"]["random_coors"].items():
    for seed, seed_dict in feature_dims_dict.items():


        active_substrate = seed_dict["substrate"]
        evol_algorithm = create_evol_algorithm(substrate=active_substrate)
        substrate_dimensions = int(active_substrate.query_coors.shape[1]/2)
        initial_cppn_layers = config["algorithm"]["genome"]["cppn_init_hidden_layers"](active_substrate.query_coors.shape[1])

        run_id = f"no_data_random_coors_{substrate_dimensions}d"

        wanbd_name = run_id
        wandb_tags = [config["substrate"]["hidden_layer_type"], env_name, "no_data", "random_coors", f"{config['substrate']['hidden_depth']}_hl", f"{config['algorithm']['neat']['pop_size']}pop", f"{config['environment']['backend']}"]

        wandb.init(
            name=wanbd_name,
            project="substrates",
            tags=wandb_tags,
            config=config  
        )

        wandb.config.update(
            {
                "substrate": {
                    "obs_size": obs_size,
                    "act_size": act_size,
                    "num_queries": active_substrate.query_coors.shape[0],
                    "query_dim": active_substrate.query_coors.shape[1],
                    },
                "algorithm": {
                    "neat": {
                        "num_inputs": evol_algorithm.num_inputs,
                        },
                    "genome": {
                        "cppn_init_hidden_layers": initial_cppn_layers,
                        },
                    },
                "random_coor_seed": seed,
            },
        )
        
        genome_save_dir = os.path.join(OUTPUT_DIR, "genome", run_id)
        os.makedirs(genome_save_dir, exist_ok=True)

        pipeline = CustomPipeline(
            algorithm=evol_algorithm,
            problem=env_problem,
            seed=int(seed),
            generation_limit=config["pipeline"]["generation_limit"],
            fitness_target=config["pipeline"]["fitness_target"],
            is_save=True,
            save_dir=genome_save_dir,
        )

        init_state = pipeline.setup()
        train_state, best_genome = pipeline.auto_run(state=init_state)

        print(f"\nTraining finished. Best fitness achieved: {pipeline.best_fitness}")

        wandb.finish()

        append_summary_row(
            path=f"{OUTPUT_DIR}/summary_results.csv",
            row={
                "dimensionality": int(active_substrate.query_coors.shape[1] / 2),
                "sampling": "no_data",
                "method": "random_coors",
                "max_fitness": float(pipeline.best_fitness),
            },
        )

        # Uses the trained state for parameters, but overrides the key for reset/render. This way all videos start in the same state for comparison.
        k_show = jax.random.PRNGKey(config["experiment"]["seed"]+5)
        show_state = train_state.update(randkey=k_show)

        # Built-in show method to produce and save a video of the agent
        pipeline.show(
            state=show_state,
            best=best_genome,
            output_type="mp4",
            save_path=f"{OUTPUT_DIR}/video/agent_no_data_random_coors_{feature_dims_label}_seed{seed}.mp4",
        )

        # All coordinates are needed for visualization
        input_coors = seed_dict["input_coors"]
        hidden_coors = seed_dict["hidden_coors"]
        output_coors = seed_dict["output_coors"]

        # Visualizes the CPPN
        visualize_cppn(
            pipeline=pipeline, 
            state=train_state, 
            save_path=f"{OUTPUT_DIR}/topology/cppn_no_data_random_coors_{feature_dims_label}_seed{seed}.svg"
        )
        
        # Visualizes a representation of the neural network in 2D space
        visualize_nn(
            pipeline=pipeline, 
            state=train_state, 
            save_path=f"{OUTPUT_DIR}/topology/nn_no_data_random_coors_{feature_dims_label}_seed{seed}.svg", 
            substrate=active_substrate, 
            input_coors=input_coors, 
            hidden_coors=hidden_coors, 
            output_coors=output_coors, 
            hidden_depth=config["substrate"]["hidden_depth"], 
            max_weight=config["algorithm"]["hyperneat"]["max_weight"], 
        )

save to output_evolution/genome/no_data_random_coors_2d
initializing
initializing finished
start compile
compile finished, cost time: 29.897883s
Generation: 1, Cost time: 4162.63ms
 	fitness: valid cnt: 500, max: 51.8829, min: -3964.4988, mean: -1557.1019, std: 1287.4210

	node counts: max: 8, min: 6, mean: 7.27
 	conn counts: max: 12, min: 5, mean: 10.34
 	species: 10, [77, 72, 1, 40, 40, 17, 20, 1, 2, 230]

Generation: 2, Cost time: 4134.41ms
 	fitness: valid cnt: 500, max: 51.9270, min: -3952.5686, mean: -418.3173, std: 907.4247

	node counts: max: 9, min: 6, mean: 7.45
 	conn counts: max: 14, min: 4, mean: 10.74
 	species: 10, [14, 70, 1, 56, 61, 56, 43, 14, 9, 176]

Generation: 3, Cost time: 4174.62ms
 	fitness: valid cnt: 500, max: 51.5240, min: -3902.7146, mean: -134.7129, std: 610.2314

	node counts: max: 10, min: 5, mean: 7.56
 	conn counts: max: 15, min: 0, mean: 10.83
 	species: 10, [25, 64, 1, 57, 62, 51, 65, 18, 12, 145]

Generation: 4, Cost time: 4067.96ms
 	fitness: vali

compute_ms,▁▁▁▁▂▅▃▂▂▂▃▃▃▃▃▃▅▄▄▄▄▄▅▅▅▆▅▆▅▅▆▆▆▆▆▇▇▇▇█
compute_ms_pop,▁▁▄▂▂▃▃▃▃▃▃▃▄▄▄▅▄▄▄▄▄▄▅▅▅▄▅▅▅▅▆▅▅▅▆▇▇█▇▇
fitness_max,▁▁▁▁▁▂▂▂▄▃▃▃▃▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▃▄▇██▇▇█▇█▇▇
fitness_mean,▃▇▇▅▃▄▆▅▃▅▇▇▆▆▇▆▇▅█▆██▇▇▇▇██▇▆▆▅▄▄▄▂▂▄▁▃
fitness_min,▆▅▁▁▁▄▁▁▂▄▁▁▅▅▂█▁▄▁▅▄▁▄█▄▄▃▁▄▄▁▃█▁▂▁▆▅▁▁
fitness_std,█▃▂▂▂▂▂▂▂▂▂▁▁▂▁▁▁▁▁▂▁▁▁▁▂▂▁▁▂▂▂▃▃▃▃▃▃▃▂▂
generation,▁▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▅▅▅▆▆▆▆▇▇▇▇▇█
num_species,████▇█▄▂▁▃▅▄▄▃▃▂▂▂▃▂▂▂▂▃▂▃▂▂▁▁▁▁▁▂▁▁▁▁▁▁
compute_ms,4479.05087
compute_ms_pop,8.9581
fitness_max,317.40295


2025-11-07 08:06:05.625925: W external/xla/xla/service/gpu/autotuning/dot_search_space.cc:200] All configs were filtered out because none of them sufficiently match the hints. Maybe the hints set does not contain a good representative set of valid configs?Working around this by using the full hints set instead.
2025-11-07 08:06:05.625942: W external/xla/xla/service/gpu/autotuning/dot_search_space.cc:200] All configs were filtered out because none of them sufficiently match the hints. Maybe the hints set does not contain a good representative set of valid configs?Working around this by using the full hints set instead.


Total reward:  410.1238


/home/andi/anaconda3/envs/jax/lib/python3.10/subprocess.py:1796: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = _posixsubprocess.fork_exec(


...

save to output_evolution/genome/no_data_random_coors_20d
initializing
initializing finished
start compile
compile finished, cost time: 32.327463s
Generation: 1, Cost time: 4381.48ms
 	fitness: valid cnt: 500, max: 50.9756, min: -3955.7932, mean: -1139.1832, std: 1200.5061

	node counts: max: 44, min: 42, mean: 43.28
 	conn counts: max: 84, min: 41, mean: 81.13
 	species: 10, [25, 73, 76, 35, 23, 26, 27, 11, 12, 192]

Generation: 2, Cost time: 4353.12ms
 	fitness: valid cnt: 500, max: 55.2534, min: -3948.1084, mean: -456.3572, std: 1022.8566

	node counts: max: 45, min: 41, mean: 43.53
 	conn counts: max: 86, min: 1, mean: 77.97
 	species: 10, [86, 13, 2, 36, 38, 1, 28, 36, 2, 258]

Generation: 3, Cost time: 4366.00ms
 	fitness: valid cnt: 500, max: 55.2207, min: -3949.2139, mean: -176.7495, std: 674.2286

	node counts: max: 46, min: 42, mean: 43.88
 	conn counts: max: 87, min: 41, mean: 83.13
 	species: 10, [87, 47, 55, 49, 1, 40, 32, 14, 9, 166]

Generation: 4, Cost time: 4338.36ms
 	

compute_ms,▁▁▁▂▃▂▂▂▃▃▃▄▄▄▅▅▅▅▅▆▆▆▇▆▇█▇█▇█▇███▇███▇█
compute_ms_pop,▁▂▂▂▂▃▃▃▃▃▃▃▃▄▄▄▅▅▅▅▇▇▇▇▇▇▇███▇█▇▇▇███▇▇
fitness_max,▁▁▁▂▄▅▅▄▅▅▆▆▆▆▆▆▆▆▇▇▇▇▇▆▇█▇▆▇█▇▇▇▇▇▇▇▇▇▇
fitness_mean,▂▁▃▂▂▃▃▄▅▅▆▆▅█▇▇▇█▇▇▇██▆▆▇▆▇▇▆▇▇▇▇▇▇▇▇██
fitness_min,▃▁▁▃▄▁▄▁▁▃▄▄▅▃▃▁▃▄▆▃▃█▆▇▁▄▃█▆▄▃▃▅▁█▃▄▁▃▃
fitness_std,█▇█▆▄▆▃▄▅▁▃▄▅▄▂▃▂▇█▆▄▄▃▄▇▂▄▃▄▃▄▇▅▃▄▅▃▄▄▄
generation,▁▁▁▁▂▂▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇▇▇▇█
num_species,█████▇▇▇██▃▄▂▂▂▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
compute_ms,4715.52992
compute_ms_pop,9.43106
fitness_max,613.88409


Total reward:  712.3747


/home/andi/anaconda3/envs/jax/lib/python3.10/subprocess.py:1796: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = _posixsubprocess.fork_exec(


mp4 saved to:  output_evolution/video/agent_no_data_random_coors_19_seed10319.mp4
Visualizing CPPN. Saving to output_evolution/topology/cppn_no_data_random_coors_19_seed10319.svg.
Manually reconstructing the phenotype. A visual layout will be generated.
Substrate has 1792 potential connections.
Visualizing 1689 connections. Excluded loops. Weight threshold: 0.005
Visualization saved to: output_evolution/topology/nn_no_data_random_coors_19_seed10319.svg
